In [1]:
import cv2
import torch
import numpy as np

## 1. Load YOLO Object Detector

In [2]:
# Cargar el modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Definir clases de interés
CLASSES_OF_INTEREST = ['car']

Using cache found in C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-22 Python-3.9.7 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


## 2. Define Tracker Class (per centroides)

In [3]:
import numpy as np
from scipy.spatial.distance import cdist

class Tracker:
    MAX_DISAPPEAR_LIMIT = 5
    def __init__(self):
        self.next_unique_id = 0
        self.trackers = {}
        self.disappear_trackers = {}
        self.tracked_bboxes = {}
    
    
    def init_object(self,centroid,boxes):
        global next_unique_id
        self.trackers[self.next_unique_id] = centroid
        self.tracked_bboxes[self.next_unique_id] = boxes
        self.disappear_trackers[self.next_unique_id] = 0
        self.next_unique_id+=1

    def del_object(self,track_id):
        del self.trackers[track_id]
        del self.tracked_bboxes[track_id]
        del self.disappear_trackers[track_id]

    def update_object(self,bboxes):
        
        if(len(bboxes)==0):
            
            for oid in list(self.disappear_trackers.keys()):
                self.disappear_trackers[oid]+=1
                
                if self.disappear_trackers[oid] > Tracker.MAX_DISAPPEAR_LIMIT:
                    self.del_object(oid)
                
            return self.tracked_bboxes
        
        else:   
            input_centroids = np.zeros((len(bboxes),2)) 
            for i in range(len(bboxes)):
                x,y,w,h = bboxes[i][0],bboxes[i][1],bboxes[i][2],bboxes[i][3]
                cx,cy = x + w/2 , y + h/2
                input_centroids[i] = (cx,cy)

            
            if(len(self.trackers)==0):
                for i in range(len(input_centroids)):
                    self.init_object(input_centroids[i],bboxes[i])
            
            else:
                
                tracker_centroids = list(self.trackers.values())

                distance_matrix = cdist(np.array(tracker_centroids) , input_centroids)

                rows = distance_matrix.min(axis=1).argsort()
                cols = distance_matrix.argmin(axis=1)[rows]

                usedRows = set()
                usedCols = set()
                
                tracker_ids = list(self.trackers.keys()) 
                for row,col in zip(rows,cols):
                    if row in usedRows or col in usedCols:
                        continue
                    track_id = tracker_ids[row]
                    
                    self.trackers[track_id] = input_centroids[col]
                    self.tracked_bboxes[track_id] = bboxes[col]

                    self.disappear_trackers[track_id] = 0
                    usedRows.add(row)                                
                    usedCols.add(col)
                unusedRows = set(range(0,distance_matrix.shape[0])).difference(usedRows)
                unusedCols = set(range(0,distance_matrix.shape[1])).difference(usedCols)
                if(distance_matrix.shape[0]>=distance_matrix.shape[1]):
                    
                    for r in unusedRows: 
                        track_id = tracker_ids[r]
                        self.disappear_trackers[track_id]+=1
                        if(self.disappear_trackers[track_id] > Tracker.MAX_DISAPPEAR_LIMIT):
                            self.del_object(track_id)
                else:
                    for c in unusedCols:                    
                        self.init_object(input_centroids[c],bboxes[c])

        return self.tracked_bboxes

## 3. Load Video

In [4]:
# Create tracker object
tracker2 = Tracker()

cap = cv2.VideoCapture("/Users/mirvi/Desktop/mii/UAB/4.1/PSIV2/detect mateicules/repte2_psiv2/data_r2/short_uab_flow.mp4")

## 4. Object Detection and Tracking over video

In [5]:
import time

In [6]:
# PROCESS VIDEO IN GRAYSCALE

gray_video_og = []

while True:
    ret, fr = cap.read()
    if not ret:
        break
    gray_video_og.append(cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY))

In [7]:
i = 0
while True:
    cv2.imshow('frame', gray_video_og[i])
    i+=1
    k = cv2.waitKey(30)
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

: 

In [89]:
import time
import cv2

# Define reference line (in the middle of the ROI)
line_y_position = (820 - 350) // 2  # Adjust according to the height of the ROI

# copy gray video
gray_video = gray_video_og.copy()

# Counters for cars moving up and down
count_up = 0
count_down = 0

# Threshold to detect significant position change
movement_threshold = 5

# Store previous positions of each car
previous_positions = {}

frame_id = 0

while True:
    # Start time measurement
    time_start = time.time()

    # # Get frame
    # ret, frame = cap.read()
    # if not ret:
    #     break

    # # to gray
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame = gray_video[frame_id]

    height, width = frame.shape

    # Extract ROI
    roi = frame[350:820, 100:520]

    # 1. Object Detection
    ts_od = time.time()
    results = model(roi)

    detections = []
    for det in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = det

        if model.names[int(cls)] == 'car':
            w = int(x2 - x1)
            h = int(y2 - y1)
            x = int(x1)
            y = int(y1)
            detections.append([x, y, w, h])

    te_od = time.time()

    cv2.putText(frame, f'ObjDet time: {(te_od - ts_od):.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(frame, f"UP: {count_up}", (10, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f"DOWN: {count_down}", (10, 750), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 2. Object Tracking
    boxes_ids = tracker2.update_object(detections)

    # Draw bounding boxes and text
    for box_id, box in boxes_ids.items():
        x, y, w, h = box
        id = box_id

        # Draw ID and bounding box
        cv2.putText(roi, str(id), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 127, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Check if the car crossed the line
        current_y = y + h // 2  # Center of the car in Y-axis

        # If the car is detected for the first time
        if id not in previous_positions:
            previous_positions[id] = current_y
            continue

        previous_y = previous_positions[id]

        # Check the direction of the car's movement
        if previous_y < line_y_position and current_y > line_y_position:
            count_down += 1  # Car moved downward
        elif previous_y > line_y_position and current_y < line_y_position:
            count_up += 1  # Car moved upward

        # Update the car's previous position
        previous_positions[id] = current_y

    # Show reference line
    cv2.line(roi, (0, line_y_position), (roi.shape[1], line_y_position), (0, 255, 255), 2)

    # Calculate FPS
    time_end = time.time()
    time_per_frame = time_end - time_start
    fps = 1 / time_per_frame if time_per_frame > 0 else 0

    cv2.putText(frame, f'Processing time: {fps:.2f} FPS', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # Show frames
    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)

    # Control frame steps
    # cap.grab()
    frame_id += 2

    key = cv2.waitKey(30)
    # Escape key to exit
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models